# Qdrant的基本使用

https://qdrant.tech/documentation/tutorials/search-beginners/

## 准备

In [1]:
%%time
%%capture

!pip install qdrant-client

CPU times: user 10.4 ms, sys: 7.61 ms, total: 18 ms
Wall time: 3.94 s


In [2]:
%%time

documents = [
    {
        "name": "围城",
        "description": "主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。",
        "author": "钱钟书",
    },
    {
        "name": "故乡",
        "description": "小说讲述了主人公“我”（即鲁迅的化身）在阔别多年后回到故乡接母亲到城里生活的故事。在故乡，他遇到了童年的玩伴闰土和老仆人杨二嫂。通过与他们的交谈和观察，主人公感受到故乡的变化和人们生活的困苦。",
        "author": "鲁迅",
    },
    {
       "name": "阿Q正传",
        "description": "讲述了阿Q这个贫苦农民在中国封建社会中的悲惨生活。他虽然穷困潦倒，但心态自负，总是以精神胜利法来安慰自己，逃避现实的困境。然而，随着社会动荡和革命的到来，阿Q的命运变得更加悲惨，最终被误认为是革命党人而被处死。",
        "author": "鲁迅",
    }, 
]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [3]:
%%time

# 嵌入模型

from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('/models/bge-small-zh-v1.5')

CPU times: user 2.37 s, sys: 411 ms, total: 2.78 s
Wall time: 2.4 s


In [4]:
encoder.get_sentence_embedding_dimension()

512

## 使用 Qdrant

### 设置存储位置

In [5]:
%%time

from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

CPU times: user 902 ms, sys: 24.9 ms, total: 927 ms
Wall time: 928 ms


### 创建集合

In [6]:
%%time

from qdrant_client import models

collection_name="my_books"

if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
            distance=models.Distance.COSINE,
        ),
    )

CPU times: user 620 µs, sys: 87 µs, total: 707 µs
Wall time: 705 µs


### 向集合传入数据

In [7]:
%%time

client.upload_points(
    collection_name="my_books",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["description"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

CPU times: user 341 ms, sys: 51.6 ms, total: 393 ms
Wall time: 434 ms


### 查询

In [8]:
%%time

hits = client.search(
    collection_name="my_books",
    query_vector=encoder.encode("方鸿渐").tolist(),
    limit=3,
)

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': '围城', 'description': '主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。', 'author': '钱钟书'} score: 0.6095981240348884
{'name': '故乡', 'description': '小说讲述了主人公“我”（即鲁迅的化身）在阔别多年后回到故乡接母亲到城里生活的故事。在故乡，他遇到了童年的玩伴闰土和老仆人杨二嫂。通过与他们的交谈和观察，主人公感受到故乡的变化和人们生活的困苦。', 'author': '鲁迅'} score: 0.2970311065063162
{'name': '阿Q正传', 'description': '讲述了阿Q这个贫苦农民在中国封建社会中的悲惨生活。他虽然穷困潦倒，但心态自负，总是以精神胜利法来安慰自己，逃避现实的困境。然而，随着社会动荡和革命的到来，阿Q的命运变得更加悲惨，最终被误认为是革命党人而被处死。', 'author': '鲁迅'} score: 0.2725128023510518
CPU times: user 10 ms, sys: 242 µs, total: 10.3 ms
Wall time: 9.44 ms


### 过滤条件

In [9]:
%%time

hits = client.search(
    collection_name="my_books",
    query_vector=encoder.encode("方鸿渐").tolist(),
    query_filter=models.Filter(
        must=[models.FieldCondition(key="author", match=models.MatchValue(value="钱钟书"),)]
    ),
    limit=3,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': '围城', 'description': '主人公方鸿渐留学回国后，面临找工作和个人感情的种种问题。他经历了几段感情波折，包括与鲍小姐的失败婚姻和与孙柔嘉的恋情，最终与孙柔嘉结婚。但婚后生活并不如意，他在事业上也遭遇挫折，未能实现自己的理想。', 'author': '钱钟书'} score: 0.6095981240348884
CPU times: user 3.83 ms, sys: 3.34 ms, total: 7.16 ms
Wall time: 6.42 ms
